In [1]:
import random
import glob
from scipy import ndimage
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [ ]:
def show_patches(patches, rows, cols, title='Patches', size=10):
    assert len(patches) == rows * cols
    fig, ax = plt.subplots(
        nrows=rows, ncols=cols, figsize=(size, size / cols * rows),
        subplot_kw={'xticks': [], 'yticks': []})
    fig.suptitle(title)
    for x, y in itertools.product(range(rows), range(cols)):
        patch = patches[x * cols + y]
        ax[x, y].get_xaxis().set_visible(False)
        ax[x, y].imshow(patch, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.show()

def unpack_tiles(image, size=(10, 20)):
    grid = image.shape[0] // size[0], image.shape[1] // size[1]
    tiles = np.empty(grid + size)
    for x, y in np.ndindex(grid):
        left, top = x * size[0], y * size[1]
        tiles[x, y] = image[left: left + size[0], top: top + size[1]]
    tiles = tiles.reshape((tiles.shape[0], -1))
    return tiles, grid

def pack_tiles(tiles, grid, size=(10, 20)):
    tiles = tiles.reshape(grid + size)
    image = np.empty((grid[0] * size[0], grid[1] * size[1]))
    for x, y in np.ndindex(grid):
        left, top = x * size[0], y * size[1]
        image[left: left + size[0], top: top + size[1]] = tiles[x, y]
    return image

def compute_pcs(data):
    cov = np.cov(data.T)
    eig, pcs = np.linalg.eig(cov)
    order = np.argsort(eig)[::-1]
    pcs, eig = pcs[order], eig[order]
    pcs = pcs.T
    return pcs, eig

def compress_tiles(tiles, pcs):
    mean = tiles.mean(axis=0)
    tiles -= mean
    transformed = np.dot(pcs, tiles.T)
    reconstructed = np.dot(pcs.T, transformed).T
    reconstructed += mean
    return reconstructed

In [ ]:
# Exercise 3.1
picture = ndimage.imread('imgpca/b8.jpg').astype(np.float64)
size = (10, 20)
tiles, grid = unpack_tiles(picture, size)
pcs, eig = compute_pcs(tiles)

for keep in [1, 2, 4, 8, 16, 100]:
    compressed = compress_tiles(tiles, pcs[:keep])
    plt.imshow(pack_tiles(compressed, grid, size), cmap=plt.cm.gray, interpolation='nearest')
    plt.show()